In [ ]:
!pip install vadersentiment
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 4.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 1.6 MB/s 
     |████████████████████████████████| 636 kB 18.9 MB/s 
     |████████████████████████████████| 85 kB 1.5 MB/s 
     |████████████████████████████████| 88 kB 7.7 MB/s 
     |████████████████████████████████| 5.2 MB 46.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 5.5 MB 41.3 MB/s 
     |████████████████████████████████| 1.3 MB 56.6 MB/s 
     |████████████████████████████████| 182 kB 64.6 MB/s 
     |████████████████████████████████| 7.6 MB 47.1 MB/s 
     |████████████████████████████████| 1.1 MB 36.6 MB/s 
  Created wheel for hdbscan: filename=hdbscan

In [ ]:
import pandas as pd
import numpy as np
import gensim
from nltk.stem import WordNetLemmatizer, SnowballStemmer
np.random.seed(10)
import nltk
from bertopic import BERTopic
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
import seaborn as sns
import plotly.express as px
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
def clean_text(text):
    text = text.lower() # lowercase everything
    text = text.encode('ascii', 'ignore').decode()  # remove unicode characters
    text = re.sub(r'https*\S+', ' ', text) # remove links
    text = re.sub(r'http*\S+', ' ', text)
    # cleaning up text
    text = re.sub(r'\'\w+', '', text) 
    text = re.sub(r'\w*\d+\w*', '', text)
    text = re.sub(r'\s{2,}', ' ', text)
    text = re.sub(r'\s[^\w\s]\s', '', text)
    text = re.sub("[\(\[].*?[\)\]]", "", text)
    text = re.sub('[^a-zA-Z0-9 \.]', '', text)
    return text

In [ ]:
def lemmatize(text):
  return WordNetLemmatizer().lemmatize(text, pos='v')
def preprocess(text):
    if type(text)!=str:
      return ''
    result = []
    text = text.replace('[removed]', '').replace('[deleted]', '')
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 2:
            result.append(lemmatize(token))
    return ' '.join(result)

In [ ]:
import pandas as pd
file_name = "/content/drive/MyDrive/Reddit_ToT/raw_reddit_movies_posts_comments.csv"
df = pd.read_csv(file_name)
df.head()

,id,subreddit,author,created_utc,domain,url,title,num_comments,selftext,score,comments
0,eiboup,movies,adamsandleryabish,2020-01-01 00:55:21,youtu.be,https://youtu.be/JFX-qfYbHKg,Boogie Nights (1997) - The 1979&gt;1980 New Ye...,23,NaN,1,"No, for people who have never experienced this..."
1,eicgz3,movies,FunWelcome,2020-01-01 02:05:18,self.movies,https://www.reddit.com/r/movies/comments/eicgz...,What is the most honest movie of the year?,12,"Well gang, we’ve finally passed the last conve...",1,I’ll tell ya the best movie I’ve seen this yea...
2,eickbc,movies,Courier12p,2020-01-01 02:13:50,self.movies,https://www.reddit.com/r/movies/comments/eickb...,"For what it’s worth, here’s my list for top 10...",9,"I don’t have many friends who are into movies,...",1,Thank you!...That’s a shame indeed. Incendies ...
3,eicm9l,movies,Witty-Jesus,2020-01-01 02:18:51,self.movies,https://www.reddit.com/r/movies/comments/eicm9...,Just realized “perks of being a wallflower” ap...,10,I just finished this movie for a second time a...,1,She was 15 taking (failing) her driving test a...
4,eicpsc,movies,RafaelPiros,2020-01-01 02:28:01,self.movies,https://www.reddit.com/r/movies/comments/eicps...,Can we take a moment to appreciate the great m...,9,Sometimes we (myself included) tend to complai...,1,YES!...Everything released by Denis Villeneuve...


In [ ]:
print(df.shape)
df.fillna("", inplace=True)
len(df[df.id==""])

(78925, 11)


0

In [ ]:
# df[df['selftext'].isna()] = ""
df["title"] = df["title"].astype(str) 
df["selftext"] = df["selftext"].astype(str)
df["title"] = df["title"] + " " + df["selftext"]

In [ ]:
df['cleaned_title'] = df.title.apply(preprocess)
titles = df["cleaned_title"].astype(str).to_list()

In [ ]:
topic_model = BERTopic(min_topic_size=100, n_gram_range=(1,3), nr_topics='auto', verbose=True)
topics, _ = topic_model.fit_transform(titles)

Batches:   0%|          | 0/2467 [00:00<?, ?it/s]

2022-11-29 00:59:35,590 - BERTopic - Transformed documents to Embeddings
2022-11-29 01:01:55,741 - BERTopic - Reduced dimensionality
2022-11-29 01:02:11,159 - BERTopic - Clustered reduced embeddings
2022-11-29 01:04:28,160 - BERTopic - Reduced number of topics from 107 to 77


In [ ]:
topic_model.save("/content/drive/MyDrive/Reddit_ToT/topic_movies_model")

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
topic_model.visualize_barchart(top_n_topics=4, n_words=10, height=400)

In [ ]:
freq = topic_model.get_topic_info()
freq.head(10)

,Topic,Count,Name
0,-1,34842,-1_movie_film_movies_like
1,0,21946,0_movie_film_movies_like
2,1,2286,1_batman_marvel_joker_mcu
3,2,1062,2_trailer_official trailer_official_trailers
4,3,922,3_star war_war_star_jedi
5,4,831,4_dune_villeneuve_denis_denis villeneuve
6,5,829,5_poster_posters_movie posters_official poster
7,6,826,6_star_kristen_image_kristen stewart
8,7,695,7_nolan_tenet_christopher nolan_christopher
9,8,542,8_bond_jam bond_jam_craig


In [ ]:
topic_nr = freq.iloc[3]["Topic"]  # We select a frequent topic
topic_model.get_topic(topic_nr)

[('trailer', 0.1004365849166488),
 ('official trailer', 0.05481422924293427),
 ('official', 0.05184123656940757),
 ('trailers', 0.04697277666807655),
 ('trailer netflix', 0.014901656951806766),
 ('movie trailers', 0.013742688454621566),
 ('official trailer netflix', 0.013629240938342484),
 ('teaser', 0.011871125455157106),
 ('movie', 0.011404145345950087),
 ('netflix', 0.011276572373460896)]

In [ ]:
freq["word_strength"] = freq.Topic.apply(topic_model.get_topic)
freq.head()

,Topic,Count,Name,word_strength
0,-1,34842,-1_movie_film_movies_like,"[(movie, 0.007928808156672832), (film, 0.00683..."
1,0,21946,0_movie_film_movies_like,"[(movie, 0.007596508343484919), (film, 0.00584..."
2,1,2286,1_batman_marvel_joker_mcu,"[(batman, 0.01913483264384149), (marvel, 0.011..."
3,2,1062,2_trailer_official trailer_official_trailers,"[(trailer, 0.1004365849166488), (official trai..."
4,3,922,3_star war_war_star_jedi,"[(star war, 0.02096338461886313), (war, 0.0179..."


In [ ]:
freq.to_csv("movies_topic_frequency.csv", index=False)

In [ ]:
df['created_utc'] = pd.to_datetime(df['created_utc'])
df["created_dt_month"] = df['created_utc'].dt.strftime('%m-%Y')
titles = df["cleaned_title"].astype(str).to_list()
dates = df['created_utc']
topics_over_time = topic_model.topics_over_time(docs=titles,  
                                                # topics=topics, 
                                                timestamps=dates, 
                                                global_tuning=True, 
                                                evolution_tuning=True, 
                                                nr_bins=30)

30it [1:56:34, 233.14s/it]


In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)

In [ ]:
topics_over_time.to_csv("topics_over_time_movies.csv", index=False)
representative_docs = topic_model.get_representative_docs()

docs_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in representative_docs.items() ])).T
docs_df.index.name = "topic_nr"
docs_df.to_csv("representative_docs_movies.csv")
docs_df.head()

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
topic_nr,,,,,,,,,,,,,,,,,,,,,
0,favorite fun happy movie come years prefer mov...,best year movies new millennium regard best ye...,greatest year movie history,adaptation far ballsiest premise movie see spo...,greatest showman attract lot hate catch guard ...,jack nicholson movies probably haven see face ...,best cast director work today francine maisler...,directors time basically title say directors t...,peter lamont british art director set decorato...,horror creature impreg movie wonder know movie...,...,breakfast club hit theatres day hollywood repo...,good movies detriment franchise oftentimes ada...,worse book adaptation world war like wwz book ...,film think fact better book counterparts exist...,look comedy want watch comedy movie want dumb ...,comedies musical number look comedies unexpect...,periscope hysterical havent watch movie years ...,proximity distance michael haneke funny game r...,goldeneye nintendo offer better overall experi...,lara croft tomb raider watch movie glorious dv...
32,blade runner damn masterpiece shame people hav...,blade runner think movie get oscar best pictur...,blade runner absolutely adore movie watch seco...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,rank pixar movies release date movies order re...,time pixar start remake old movies pixar relea...,pixar movies change enjoy turn red feel like f...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,cool true romance quentin tarantino masterpiec...,quentin tarantino lot feet lot good directors ...,quentin tarantino movies,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,good space thrillers fall love movies like gra...,small stake sci movies start go backlog movies...,look movies set space probably haven see invar...,george lucas special editions prequels sequels...,shall win know star war revolutionize cinema h...,episodes star war fast difficult relate charac...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df['topic'] = df['cleaned_title'].apply(lambda x: topic_model.find_topics(x)[0][0])
df.head()

,id,subreddit,author,created_utc,domain,url,title,num_comments,selftext,score,comments,cleaned_title,created_dt_month,topic
0,eiboup,movies,adamsandleryabish,2020-01-01 00:55:21,youtu.be,https://youtu.be/JFX-qfYbHKg,Boogie Nights (1997) - The 1979&gt;1980 New Ye...,23,,1,"No, for people who have never experienced this...",boogie nights new years party little death,01-2020,34
1,eicgz3,movies,FunWelcome,2020-01-01 02:05:18,self.movies,https://www.reddit.com/r/movies/comments/eicgz...,What is the most honest movie of the year? Wel...,12,"Well gang, we’ve finally passed the last conve...",1,I’ll tell ya the best movie I’ve seen this yea...,honest movie year gang finally pass conversati...,01-2020,0
2,eickbc,movies,Courier12p,2020-01-01 02:13:50,self.movies,https://www.reddit.com/r/movies/comments/eickb...,"For what it’s worth, here’s my list for top 10...",9,"I don’t have many friends who are into movies,...",1,Thank you!...That’s a shame indeed. Incendies ...,worth list decade friends movies conversations...,01-2020,13
3,eicm9l,movies,Witty-Jesus,2020-01-01 02:18:51,self.movies,https://www.reddit.com/r/movies/comments/eicm9...,Just realized “perks of being a wallflower” ap...,10,I just finished this movie for a second time a...,1,She was 15 taking (failing) her driving test a...,realize perk wallflower appeal sexist bias chi...,01-2020,53
4,eicpsc,movies,RafaelPiros,2020-01-01 02:28:01,self.movies,https://www.reddit.com/r/movies/comments/eicps...,Can we take a moment to appreciate the great m...,9,Sometimes we (myself included) tend to complai...,1,YES!...Everything released by Denis Villeneuve...,moment appreciate great movies decade include ...,01-2020,53


In [ ]:
df['cleaned_comments'] = df['comments'].map(preprocess)
df.to_csv("/content/drive/MyDrive/Reddit_ToT/reddit_movies_data_with_topics.csv", index=False)
df.head()

,id,subreddit,author,created_utc,domain,url,title,num_comments,selftext,score,comments,cleaned_title,created_dt_month,topic,cleaned_comments
0,eiboup,movies,adamsandleryabish,2020-01-01 00:55:21,youtu.be,https://youtu.be/JFX-qfYbHKg,Boogie Nights (1997) - The 1979&gt;1980 New Ye...,23,,1,"No, for people who have never experienced this...",boogie nights new years party little death,01-2020,34,people experience moment watch film time years...
1,eicgz3,movies,FunWelcome,2020-01-01 02:05:18,self.movies,https://www.reddit.com/r/movies/comments/eicgz...,What is the most honest movie of the year? Wel...,12,"Well gang, we’ve finally passed the last conve...",1,I’ll tell ya the best movie I’ve seen this yea...,honest movie year gang finally pass conversati...,01-2020,0,tell best movie see year richard jewell dunno ...
2,eickbc,movies,Courier12p,2020-01-01 02:13:50,self.movies,https://www.reddit.com/r/movies/comments/eickb...,"For what it’s worth, here’s my list for top 10...",9,"I don’t have many friends who are into movies,...",1,Thank you!...That’s a shame indeed. Incendies ...,worth list decade friends movies conversations...,01-2020,13,thank shame incendies amaze wail prove enterta...
3,eicm9l,movies,Witty-Jesus,2020-01-01 02:18:51,self.movies,https://www.reddit.com/r/movies/comments/eicm9...,Just realized “perks of being a wallflower” ap...,10,I just finished this movie for a second time a...,1,She was 15 taking (failing) her driving test a...,realize perk wallflower appeal sexist bias chi...,01-2020,53,take fail drive test turn movie spinoff senior...
4,eicpsc,movies,RafaelPiros,2020-01-01 02:28:01,self.movies,https://www.reddit.com/r/movies/comments/eicps...,Can we take a moment to appreciate the great m...,9,Sometimes we (myself included) tend to complai...,1,YES!...Everything released by Denis Villeneuve...,moment appreciate great movies decade include ...,01-2020,53,yes release denis villeneuve shadow favourites...


In [ ]:
sentiment_analyzer = SentimentIntensityAnalyzer()
df["comment_sentiment"] = df['cleaned_comments'].apply(lambda x: sentiment_analyzer.polarity_scores(x)["compound"])

In [ ]:
df['created_utc'] = pd.to_datetime(df['created_utc'])
df["created_dt_month"] = df['created_utc'].dt.strftime('%m-%Y')

In [ ]:
sentiment_df = df[["created_dt_month", "topic", "comment_sentiment"]].groupby(["created_dt_month", "topic"]).mean()
sentiment_df.head()

comment_sentiment
created_dt_month topic                   
01-2020          -1              0.431767
                  0              0.415649
                  1              0.700272
                  2              0.546107
                  3              0.436274

In [ ]:
sentiment_df = sentiment_df.reset_index()

In [ ]:
map_topic = freq[["Topic", "Name"]]
map_topic.columns = ['topic', 'name']
merged_df = sentiment_df.merge(map_topic, on=['topic'])

In [ ]:
merged_df["created_dt_month"] = pd.to_datetime(merged_df["created_dt_month"])
merged_df = merged_df.sort_values("created_dt_month")
merged_df = merged_df[merged_df['topic']!=-1]
merged_df.head()

,created_dt_month,topic,comment_sentiment,name
1035,2020-01-01,38,0.571294,38_samurai_japanese_kurosawa_akira
1061,2020-01-01,39,0.858565,39_ridley_ridley scott_scott_duel
163,2020-01-01,5,0.581398,5_poster_posters_movie posters_official poster
1087,2020-01-01,40,0.582621,40_rocky_rambo_stallone_creed
1113,2020-01-01,41,0.718242,41_black widow_widow_black_scarlett johansson


In [ ]:
plot_df = merged_df[merged_df['topic'].isin([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])]
# plot_df["created_dt_month"] = pd.to_datetime(plot_df["created_dt_month"])
fig = px.line(plot_df, x="created_dt_month", y="comment_sentiment", color="name")
fig.show()
# sns.lineplot(data=plot_df, x="created_dt_month", y="comment_sentiment", hue="name")

In [ ]:
merged_df.to_csv("movies_comment_sentiment.csv", index=False)

In [ ]:
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
docs = titles
# topic_model = BERTopic(verbose=True, n_gram_range=(1, 3),nr_topics='auto',min_topic_size=100)
# topics, _ = topic_model.fit_transform(docs)

# Preprocess Documents
documents = pd.DataFrame({"Document": docs,
                          "ID": range(len(docs)),
                          "Topic": topics})
documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
cleaned_docs = topic_model._preprocess_text(documents_per_topic.Document.values)

# Extract vectorizer and analyzer from BERTopic
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
               for topic in range(len(set(topics))-1)]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_v')
coherence = coherence_model.get_coherence()
print(coherence)

0.43920018518373266
